In [1]:
import numpy as np
import random
import pandas as pd
import gzip
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
n = 40428967  #total number of records in the clickstream data 
sample_size = 1000000
skip_values = sorted(random.sample(range(1,n), n-sample_size))
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
with gzip.open('avazu/train.gz') as f:
    train = pd.read_csv(f, parse_dates = ['hour'], date_parser = parse_date, skiprows = skip_values)

In [3]:
train = train.rename(columns={'hour': 'date'})

In [4]:
#which hours have the most clicks 
train['hour'] = train.date.apply(lambda x: x.hour)
train['day_of_week'] = train['date'].apply(lambda val: val.weekday_name)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
train.head()

,id,click,date,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C14,C15,C16,C17,C18,C19,C20,C21,hour,day_of_week
0,10000918755742328737,0,2014-10-21,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,20632,320,50,2374,3,39,-1,23,0,Tuesday
1,10007446479189647526,0,2014-10-21,1005,0,6ec06dbd,d262cf1e,f66779e6,ecad2386,7801e8d9,...,19870,320,50,2271,0,687,100075,48,0,Tuesday
2,10010485868773711631,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,15701,320,50,1722,0,35,100084,79,0,Tuesday
3,10012212068904346443,0,2014-10-21,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,...,20352,320,50,2333,0,39,-1,157,0,Tuesday
4,10017640116993553715,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,15707,320,50,1722,0,35,100084,79,0,Tuesday


In [6]:
target = train['click']
analysis = train
analysis.drop('date', axis=1, inplace=True)
analysis.drop('id', axis=1, inplace=True)
analysis.drop('click', axis=1, inplace=True)

In [7]:
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self
analysis = convert_obj_to_int(analysis)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(analysis, target, test_size = 0.2, random_state =0)
X_train.head()

,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,...,site_id_int,site_domain_int,site_category_int,app_id_int,app_domain_int,app_category_int,device_id_int,device_ip_int,device_model_int,day_of_week_int
760648,1005,0,1,0,23144,320,50,2665,0,35,...,8629950133425947508,111604357524640377,-2414393451864678829,-8826780648358443775,-6327619399419000028,7897939898949762738,-8218821468724886588,8676302243308813647,-6152820222596947464,-4227466896491634454
79333,1005,0,1,0,19733,320,50,2260,0,171,...,8629950133425947508,111604357524640377,-2414393451864678829,-8293533375951253635,-3995970211500157311,7598674530867915909,-3518045965299665547,8118924319101074504,5520661982104070019,-4227466896491634454
771344,1005,0,1,0,16615,320,50,1863,3,39,...,-8391469780661372980,-6321313078051792942,-6827508670936997654,-3459646385640828756,8511753678100510838,-3690605714457023166,-3518045965299665547,-4336882271328850164,-2827144088261068576,-4227466896491634454
179102,1005,0,1,0,15705,320,50,1722,0,35,...,1586783940683276736,-6461364947929047160,-6619379710939434495,-3459646385640828756,8511753678100510838,-3690605714457023166,-3518045965299665547,-7552201783185590451,-6351624509581666973,7099394789598017869
75318,1005,0,1,0,21761,320,50,2502,0,35,...,8629950133425947508,111604357524640377,-2414393451864678829,9153448722038021357,-6327619399419000028,7897939898949762738,-2665019263022025392,-5907353500166286839,-7178612361919215944,-4227466896491634454


In [ ]:
#randomforest with hyperparameters 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

rf_hyper = RandomForestClassifier(n_estimators = 700, max_depth = 80, max_features = 'sqrt', 
                                  min_samples_leaf = 1, min_samples_split = 50, random_state = 42)

# Train the model on training data
rf_hyper.fit(X_train, y_train);

# Use the forest's predict method on the test data
predictions = rf_hyper.predict(X_test)


scores = cross_val_score(rf_hyper, X_train, y_train)
print ("Score: ")
print scores.mean()

pd.crosstab(y_test,predictions, rownames=['True'], colnames=['Predicted'], margins=True)

print ("Log Loss: ")
print log_loss(predictions, y_test)

In [13]:
#Naive Bayes Model

from sklearn.naive_bayes import GaussianNB
NB_clf = GaussianNB()
NB_clf.fit(X_train, y_train)

NB_clf_pred = NB_clf.predict(X_test)

NB_scores = cross_val_score(NB_clf, X_train, y_train)
print ("Score: ")
print NB_scores.mean()

print ("Log Loss: ")
print log_loss(NB_clf_pred, y_test)



Score: 
0.7730752450227185
Log Loss: 
8.00792753065937


In [ ]:
#Naive Bayes Model

from sklearn.naive_bayes import GaussianNB
NB_clf = GaussianNB()
NB_clf.fit(X_train, y_train)

NB_clf_pred = NB_clf.predict(X_test)

NB_scores = cross_val_score(NB_clf, X_train, y_train)
print ("Score: ")
print NB_scores.mean()

print ("Log Loss: ")
print log_loss(NB_clf_pred, y_test)



In [14]:
#Support Vector Machine Model
'''
from sklearn.svm import SVC
SVM_clf = SVC(gamma='auto')
SVM_clf.fit(X_train, y_train) 

SVM_clf_pred = SVM_clf.predict(X_test)

SVM_scores = cross_val_score(SVM_clf, X_train, y_train)
print ("Score: ")
print SVM_scores.mean()

print ("Log Loss: ")
print log_loss(SVM_clf_pred, y_test)
'''

'\nfrom sklearn.svm import SVC\nSVM_clf = SVC(gamma=\'auto\')\nSVM_clf.fit(X_train, y_train) \n\nSVM_clf_pred = SVM_clf.predict(X_test)\n\nSVM_scores = cross_val_score(SVM_clf, X_train, y_train)\nprint ("Score: ")\nprint SVM_scores.mean()\n\nprint ("Log Loss: ")\nprint log_loss(SVM_clf_pred, y_test)\n'

In [15]:
#KNN Analysis

from sklearn.neighbors import KNeighborsClassifier

KNN_clf = KNeighborsClassifier(n_neighbors=2)
KNN_clf.fit(X_train, y_train) 

KNN_clf_pred = KNN_clf.predict(X_test)

KNN_scores = cross_val_score(KNN_clf, X_train, y_train)
print ("Score: ")
print KNN_scores.mean()

print ("Log Loss: ")
print log_loss(KNN_clf_pred, y_test)


Score: 
0.81175003070142
Log Loss: 
6.641937714780262
